In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import requests
import random
from bs4 import BeautifulSoup
import scipy.optimize as sco
import re
import pandas as pd
import scipy.optimize as opt

In [32]:
# #爬蟲取得加權指數資料
# start_date = '2018-01-01'
# end_date = '2021-12-31'

# resp = requests.get(f'https://api.finmindtrade.com/api/v3/data?dataset=TaiwanStockPrice&date={start_date}&end_date={end_date}&stock_id=TAIEX')
# TAIEX_data = resp.json()['data']
# TAIEX_data = pd.DataFrame(TAIEX_data)
# TAIEX_data['daily_return'] = TAIEX_data.close / TAIEX_data.close.shift(1) - 1

# TAIEX_data

In [49]:
#讀取上市普通股股價與加權指數資料
data = pd.read_csv('20231123103719.csv', encoding='utf-16', sep='\t')
data

,證券代碼,TSE 產業別,TEJ產業名,TSE新產業名,年月日,收盤價(元),報酬率％,市值(百萬元)
0,1101 台泥,1.0,M11A 水泥製造,M1100 水泥工業,20180102,20.4447,0.2743,155210
1,1102 亞泥,1.0,M11A 水泥製造,M1100 水泥工業,20180102,19.7346,0.0000,94793
2,1103 嘉泥,1.0,M11A 水泥製造,M1100 水泥工業,20180102,10.0545,0.7491,10421
3,1104 環泥,1.0,M11A 水泥製造,M1100 水泥工業,20180102,16.4056,0.2179,15033
4,1108 幸福,1.0,M11A 水泥製造,M1100 水泥工業,20180102,7.0625,0.3584,3400
...,...,...,...,...,...,...,...,...
1149004,9946 三發地產,14.0,M25A 建設,M2500 建材營造,20221230,11.8279,0.4115,3984
1149005,9955 佳龍,35.0,M99L 資源回收,M2335 綠能環保,20221230,19.8000,-0.5025,2044
1149006,9958 世紀鋼,10.0,M20A 金屬基本,M2000 鋼鐵工業,20221230,88.3187,-2.4229,20590
1149007,Y9997 報酬指數,NaN,NaN,NaN,20221230,29388.8000,0.3739,44062594


In [50]:
#找出所有產業類型
industrys = data['TSE新產業名'].unique()
#移除nan
industrys = industrys[~pd.isna(industrys)]
industrys

array(['M1100 水泥工業', 'M1200 食品工業', 'M1300 塑膠工業', 'M2500 建材營造',
       'M2200 汽車工業', 'M1400 紡織纖維', 'M9900 其他', 'M3700 運動休閒',
       'M2328 電子零組件', 'M1500 電機機械', 'M1600 電器電纜', 'M1722 生技醫療',
       'M1721 化學工業', 'M1800 玻璃陶瓷', 'M1900 造紙工業', 'M2000 鋼鐵工業',
       'M3800 居家生活', 'M2100 橡膠工業', 'M2600 航運業', 'M2325 電腦及週邊',
       'M2324 半導體', 'M2331 其他電子業', 'M2327 通信網路業', 'M2326 光電業',
       'M2329 電子通路業', 'M2330 資訊服務業', 'M2900 貿易百貨', 'M9700 油電燃氣業',
       'M2700 觀光餐旅', 'M2800 金融業', 'M2336 數位雲端', 'M2335 綠能環保',
       'W91   存託憑證'], dtype=object)

In [51]:
#計算該年度各產業在市場的比重
def get_industrys_weight(data, year):
    start_date = year + '0101'
    end_date = year + '1231'
    filtered_data = data[(data['年月日'] >= int(start_date)) & (data['年月日'] <= int(end_date))]
    industrys_marketValue = filtered_data.groupby('TSE新產業名').sum()['市值(百萬元)']
    industrys_weight = industrys_marketValue / industrys_marketValue.sum()
    return industrys_weight

In [104]:
#定義選取股票的函數，依照各產業中所佔比重選取股票

def get_n_stocks(data, year, n):
    industrys_weight = get_industrys_weight(data, year)
    #算出各產業要選幾隻股票
    n_stocks = (industrys_weight * n).astype(int)

    #找出所有產業類型
    industrys = data['TSE新產業名'].unique()
    #移除nan產業
    industrys = industrys[~pd.isna(industrys)]
    industrys

    for industry in industrys:
        if n_stocks[industry] > len(data[data['TSE新產業名'] == industry]['證券代碼'].unique()):
            print(industry, f'產業股票數量不足需求{n_stocks[industry]}隻，只有{len(data[data["TSE新產業名"] == industry]["證券代碼"].unique())}隻')
            n_stocks[industry] = len(data[data['TSE新產業名'] == industry]['證券代碼'].unique())


    while n_stocks.sum() != n:
        print('還差', n - n_stocks.sum(), '隻股票')
        industry = random.choice(industrys)
        industry_stocks_count = len(data[data['TSE新產業名'] == industry]['證券代碼'].unique())
        if industry_stocks_count < n_stocks[industry]:
            print(industry, '產業股票數量不足')
            continue
        else:
            n_stocks[industry] += 1
    print('選出的股票數量為', n_stocks.sum())
    #從上面算出的產業股票數量中，從該產業市值最大的股票開始選
    selected_stocks = []
    for industry in n_stocks.index:
        filtered_data = data[data['TSE新產業名'] == industry]
        filtered_data = filtered_data[filtered_data['年月日'] == int(year + '0102')]
        filtered_data = filtered_data.sort_values(by='市值(百萬元)', ascending=False)
        selected_stocks += list(filtered_data['證券代碼'].head(n_stocks[industry]))

    return selected_stocks
        

============================================================================================================================